# Reshape Author Affs and finalize base dataset

Let's compute some stats for display

In [ ]:
import pandas as pd
import json

import matplotlib as plt
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt


In [2]:
base_path="/Users/sijiawu/Work/Thesis/Data"
alt_input_base_path=base_path+"/021_affiliation_restructure"
subinput_base_path=base_path+"/022_author_combine_w_affils"
input_base_path=base_path+"/023_author_recon_w_affils"
output_base_path= base_path+"/024_author_consolidation_summary"

In [3]:
content_ex=['MISC',  'Discussion', 'Review', 'Review2',"Errata"]
content=['Article', 'Comment', 'Reply', 'Rejoinder']
jid=["aer","ecta","jpe","qje","res"]

In [4]:
with open(input_base_path+"/author_proc_all_w_red.json") as f: 
    data = f.read() 
proc_auths_all = json.loads(data) 

proc_auths=pd.read_pickle(input_base_path+"/auth_proc_w_aff_flat.pkl")
j_data=pd.read_pickle(subinput_base_path+"/022_merged_proc_scopus_inception_with_auth_split_2020_content_type_corrected.pkl")

In [5]:
proc_auths_all['https://www.jstor.org/stable/1804158']

{'authors': {'0': {'raw': 'Roy G. Blakey',
   'init': 'roy g. blakey',
   'auth_suffix': [],
   'a1': 'roy g. blakey',
   'a2': 'r. g. blakey',
   'a3': 'r. blakey',
   'affs': {}},
  '1': {'raw': 'Gladys C. Blakey',
   'init': 'gladys c. blakey',
   'auth_suffix': [],
   'a1': 'gladys c. blakey',
   'a2': 'g. c. blakey',
   'a3': 'g. blakey',
   'affs': {}}},
 'year': 1939,
 'content_type': 'Article',
 'jid': 'aer'}

In [6]:
aff_out=[]
issue=[]
for i in proc_auths_all.keys():
    if proc_auths_all[i]['content_type'] in content_ex:
        continue
    if proc_auths_all[i]['year']<1940:
        continue
    temp={
        "url":i,
        'year': proc_auths_all[i]["year"],
        'content_type': proc_auths_all[i]["content_type"],
        'jid': proc_auths_all[i]["jid"],
        }
    for j in proc_auths_all[i]["authors"].keys():
        # print(i)
        # print(proc_auths_all[i]["authors"][j]["resolution"])
        temp2={"auth_num": j, 'raw':proc_auths_all[i]["authors"][j]["raw"],"resolution":proc_auths_all[i]["authors"][j]["resolution"]}
        for k in proc_auths_all[i]["authors"][j]['affs'].keys():
            try:
                temp3={"aff_num":k, 'affiliation': proc_auths_all[i]["authors"][j]['affs'][k]['aff_main_final']}
                aff_out.append(temp|temp2|temp3)
            except:
                print(i)
                print('"'+proc_auths_all[i]["authors"][j]['affs'][k]['aff_main_final']+'"')
                issue.append(i)
                

In [7]:
aff_all=pd.DataFrame(aff_out)
aff_all["id"]=aff_all["url"].str.split("/").str[-1]

affs_uni_=aff_all['affiliation'].unique()
affs_uni_.sort()
len(affs_uni_)

2392

In [8]:
proc_auths["id"]=proc_auths["url"].str.split("/").str[-1]

print(proc_auths["url"].unique().shape)
print(proc_auths.shape)
display(proc_auths["content_type"].value_counts())
display(proc_auths.head())


(29382,)
(46425, 27)


content_type
Article      43420
Comment       1807
Reply         1064
Rejoinder      134
Name: count, dtype: int64

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,fl,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id
0,0,"Enke, Benjamin",benjamin enke,[],benjamin enke,b. enke,b. enke,enke,"{national bureau of economic research - nber, harvard university}",2020,...,benjamin enke,2,b,b. enke,9720,benjamin enke,NaN,,benjamin enke,qjaa012
1,0,Lauren Falcao Bergquist,lauren falcao bergquist,[],lauren f. bergquist,l. f. bergquist,l. bergquist,bergquist,{university of michigan},2020,...,lauren bergquist,3,l,l. bergquist,13075,lauren f. bergquist,NaN,,lauren f. bergquist,26966478
2,1,Michael Dinerstein,michael dinerstein,[],michael dinerstein,m. dinerstein,m. dinerstein,dinerstein,{university of chicago},2020,...,michael dinerstein,2,m,m. dinerstein,15785,michael dinerstein,NaN,,michael dinerstein,26966478
3,0,Dominic Coey,dominic coey,[],dominic coey,d. coey,d. coey,coey,{facebook},2020,...,dominic coey,2,d,d. coey,8638,dominic coey,NaN,,dominic coey,26966479
4,1,Bradley J. Larsen,bradley j. larsen,[],bradley j. larsen,b. j. larsen,b. larsen,larsen,"{national bureau of economic research - nber, stanford university}",2020,...,bradley larsen,3,b,b. larsen,2798,bradley j. larsen,NaN,bradley j. larsen,bradley j. larsen,26966479


In [9]:
res=pd.DataFrame(proc_auths["resolution"].unique()).rename(columns={0: "resolution"})
res=res.sort_values(by="resolution").reset_index(drop=True).reset_index().rename(columns={"index":"id_author"})

res.to_excel(output_base_path+"/024_consolidated_author_name_list.xlsx", index=False)

In [10]:
proc_auths_auid=pd.merge(proc_auths,res,how="left", on="resolution")
proc_auths_exp=pd.merge(aff_all,res,how="left", on="resolution")

In [11]:
proc_auths_exp.to_excel(output_base_path+"/024_auth_aff_expanded.xlsx", index=False)
proc_auths_exp.to_pickle(output_base_path+"/024_auth_aff_expanded.pkl")

In [12]:
sub_arr=['jid', 
        'url',
        'id',
        'year',
        'content_type',
        'auth_ord', 
        'raw', 
        'init',
        'init_order', 
        'suffix', 
        'a1',
        'a1_order', 
        'a2',
        'a2_order', 
        'a3', 
        'a3_order',
        'last', 
        'fl', 
        'ini', 
        'inil', 
        'affs', 
        'resolution', 
        'id_author'
        ]

proc_auths_auid[sub_arr].to_excel(output_base_path+"/024_auth_proc_w_aff_flat_sub_auid.xlsx", index=False)
proc_auths_auid[sub_arr].to_pickle(output_base_path+"/024_auth_proc_w_aff_flat_sub_auid.pkl")

# Affiliations are in a set

In [13]:
j_data.to_excel(output_base_path+"/024_merged_proc_scopus_inception_with_auth_split_2020_content_type_corrected.xlsx", index=False)
j_data.to_pickle(output_base_path+"/024_merged_proc_scopus_inception_with_auth_split_2020_content_type_corrected.pkl")

In [14]:
proc_auths_auid.head()

,auth_ord,raw,init,suffix,a1,a2,a3,last,affs,year,...,a1_tk_count,ini,inil,r_a1,r_a1_name,p_a1,p_a1_name,resolution,id,id_author
0,0,"Enke, Benjamin",benjamin enke,[],benjamin enke,b. enke,b. enke,enke,"{national bureau of economic research - nber, harvard university}",2020,...,2,b,b. enke,9720,benjamin enke,NaN,,benjamin enke,qjaa012,1406
1,0,Lauren Falcao Bergquist,lauren falcao bergquist,[],lauren f. bergquist,l. f. bergquist,l. bergquist,bergquist,{university of michigan},2020,...,3,l,l. bergquist,13075,lauren f. bergquist,NaN,,lauren f. bergquist,26966478,8632
2,1,Michael Dinerstein,michael dinerstein,[],michael dinerstein,m. dinerstein,m. dinerstein,dinerstein,{university of chicago},2020,...,2,m,m. dinerstein,15785,michael dinerstein,NaN,,michael dinerstein,26966478,9994
3,0,Dominic Coey,dominic coey,[],dominic coey,d. coey,d. coey,coey,{facebook},2020,...,2,d,d. coey,8638,dominic coey,NaN,,dominic coey,26966479,3372
4,1,Bradley J. Larsen,bradley j. larsen,[],bradley j. larsen,b. j. larsen,b. larsen,larsen,"{national bureau of economic research - nber, stanford university}",2020,...,3,b,b. larsen,2798,bradley j. larsen,NaN,bradley j. larsen,bradley j. larsen,26966479,1595
